# Evaluation
In this notebook, a list of numbers is defined to compare different implementations of the mysqrt function using cProfile.
Therefore, each implementation can be loaded from the `./versions` directory and be assessed using the `get_stats()` function

## Preparation

In [1]:
import cProfile as cp
import pstats
from pstats import SortKey
from typing import Sequence, Dict

In [2]:
def clean_stats(stats: pstats.Stats):
    '''
    Reduces an instance pstats.Stats to only function name and values for "ncalls", "cumtime", "percall", and "tottime".
    ncalls:   How often a function was called
    cumtime:  How much time was spent in the function (including calls by this function)
    percall:  How much time was spent per call (without subcalls)
    tottime:  How much time was spent in the function (without subcalls)
    '''
    cleaned = {}
    profiles = stats.get_stats_profile().func_profiles
    for func_name, func_profile in profiles.items():
        cleaned[func_name] = {'ncalls': func_profile.ncalls,
                              'cumtime': func_profile.cumtime,
                              'percall': func_profile.percall_tottime, 
                              'tottime': func_profile.tottime}
    return cleaned

def get_stats(module_name: str, num: int):
    '''
    Function that profiles the mysqrt function of a provided module following the following documentation
    https://docs.python.org/3/library/profile.html
    
    :param module_name:  Name of the module to be evaluated (Use the import name)
    :param num:          Number to evaluate the model on
    '''
    
    with cp.Profile() as profile:
        eval(module_name).mysqrt(num)
        profile.dump_stats(module_name)
        p = pstats.Stats(module_name)
        p.strip_dirs().sort_stats(SortKey.CUMULATIVE)
    return clean_stats(p)

In [3]:
def evaluate_module(module_name: str, numbers: Sequence[int]):
    '''
    Evaluate one module over all numbers 
    '''
    return [get_stats(module_name, num) for num in numbers]

def extract_summary(results: Sequence[Dict], func_name: str, attribute='tottime'):
    '''
    For each result in the results list, get the requested attribute for the requested function (fnc_name)
    '''
    return [result[func_name][attribute] for result in results]

## Comparison
Each module is loaded and compared on the same numbers:
- v1: As submitted for hw2
- v2: Vectorized `getLowUpper`
- v3: Vectorized `isperfect`
- v4: Vectorized `isperfect` and only one call of `np.sqrt` in `mysqrt()`

In [4]:
import versions.schmidt_hw2 as v1
import versions.schmidt_hw2_v2 as v2
import versions.schmidt_hw2_v3 as v3
import versions.schmidt_hw2_v4 as v4

numbers = [1234, 12345, 123456, 1234567]

In [5]:
results1 = evaluate_module("v1", numbers)

In [6]:
results2 = evaluate_module("v2", numbers)

In [7]:
results3 = evaluate_module("v3", numbers)

In [8]:
results4 = evaluate_module("v4", numbers)

Comparing results

In [9]:
cumtime1 = extract_summary(results1, 'mysqrt', 'cumtime')
cumtime2 = extract_summary(results2, 'mysqrt', 'cumtime')
cumtime3 = extract_summary(results3, 'mysqrt', 'cumtime')
cumtime4 = extract_summary(results4, 'mysqrt', 'cumtime')

In [10]:
print(cumtime1)
print(cumtime2)
print(cumtime3)
print(cumtime4)

[0.018, 0.48, 15.146, 494.165]
[0.027, 0.59, 16.939, 519.856]
[0.003, 0.006, 0.273, 7.818]
[0.002, 0.005, 0.069, 7.034]


As we can see, v2 (vectorizing `getLowUpper`) has the worst performance for all numbers while the vectorization of `isperfect` has the strongest positive effect.
Calculating `sqrt(n)` only once at not for every loop (v4), on top of the `isperfect` vectorization, leads to the best result.

In [14]:
import numpy as np

benchmark = np.array(cumtime1)
relative2 = np.array(cumtime2) / benchmark
relative3 = np.array(cumtime3) / benchmark
relative4 = np.array(cumtime4) / benchmark

In [15]:
print(relative2)
print(relative3)
print(relative4)

[1.5        1.22916667 1.11838109 1.05198871]
[0.16666667 0.0125     0.01802456 0.01582063]
[0.11111111 0.01041667 0.00455566 0.01423411]


The values in the numpy arrays show the relative time for all adaptions